In [2]:
client.restart()

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787,Cluster Workers: 1 Cores: 4 Memory: 16.81 GB


In [19]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
from dask.dot import dot_graph
import itertools
import logging
import netCDF4
import numpy as np
import dask.array as da
from dask import delayed
import time
from dask.distributed import Client
from urllib import request
from multiprocessing import Pool

client = Client('scheduler:8786')

download_location = '/temp'
data_url = 'http://nasanex.s3.amazonaws.com'
# data_url = 'http://172.21.0.1:8080'
max_download_attempts = 5

all_models = ['ACCESS1-0',  'BNU-ESM', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM-CHEM', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M', 'bcc-csm1-1', 'inmcm4']
# all_models = ['ACCESS1-0', 'BNU-ESM'] 
all_vars = ['tasmax', 'pr']
all_years = {
     # 'historical': list(range(1971, 1976))
    'historical': list(range(1971, 2001))
}

def get_dataset_url(variable, scenario, model, year, prefix = data_url):
    prefix_filename = '/'.join([prefix, 'NEX-GDDP', 'BCSD', scenario, 'day', 'atmos', variable, 'r1i1p1', 'v1.0'])
    filename = '_'.join([variable, 'day', 'BCSD', scenario, 'r1i1p1', model, str(year) + '.nc'])
    url = '/'.join([prefix_filename, filename])
    return url

def get_context(year, **kwargs):
    variables = [kwargs.get('variable')] if kwargs.get('variable') else all_vars
    scenarios = ['historical']
    models = [kwargs.get('model')] if kwargs.get('model') else all_models
    outlist = []
    combinations = list(itertools.product(variables, scenarios, models))
    result = list(map(lambda comb: [ *comb, year ], combinations))
    return result

def get_year_ensemble(year, variable = 'tasmax'):
    context = get_context(year, variable = variable)
    datasets = list(map(lambda x: str(get_dataset_url(*x)), context))
    return datasets

def download_file(url):
    print("url: " + url)
    attempts = 0
    success = False
    filename = ""
    while attempts < max_download_attempts and not success:
        time.sleep(2 ** attempts)
        filename = '/'.join([download_location, str(url.split('/')[-1])])
        print("Downloading file at " + filename)
        u = request.urlopen(url)
        f = open(filename, 'wb')
        f.write(u.read())
        f.close()
        success = True
        break
    return filename

def download_file_list(url_list):
    print("Starting download pool")
    pool = Pool()
    res = pool.map(download_file, url_list)
    print("Jobs sent")
    pool.close()
    pool.join()
    print("Downloads finished")
    print(res)
    return res

def download_and_stack(year, variable):
    dsets_urls = list(map(lambda x: get_year_ensemble(x, variable = variable), [year]))[0]
    filenames = download_file_list(dsets_urls)
    datasets = [ netCDF4.Dataset(filename) for filename in filenames ]
    dask_arrays = []
    for dset in datasets:
        dask_arrays.append(da.from_array(dset[str(variable)], chunks= (366, 144, 144)))
    try:
        final_stack = da.stack(dask_arrays, axis = 0)
    except:
        return filenames, None, None
    return filenames, datasets, final_stack

def get_stacks_mod_avg(a, chunksize):
    nmodels, time, lat, lon = a.shape
    nstacks_lat = int(np.ceil(lat / chunksize))
    nstacks_lon = int(np.ceil(lon / chunksize))
    
    stacks = []
    
    for i in range(nstacks_lat):
        for j in range(nstacks_lon):
            latmin, latmax = i * chunksize, (i+1) * chunksize
            lonmin, lonmax = j * chunksize, (j+1) * chunksize
            print(i, j, '~>', latmin, latmax, lonmin, lonmax)
            stacked = a[:, :, latmin:latmax, lonmin:lonmax]
            print(stacked)
            stacks.append(stacked)
    return stacks

def copy_dataset(src, output_filename):
    dst = netCDF4.Dataset('/home/jovyan/work/' + output_filename, "w")
    # copy attributes
    for name in src.ncattrs():
        dst.setncattr(name, src.getncattr(name))
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(
            name, (dimension)
        )
    dst.close()
    return output_filename

def restack(chunk_list, chunksize):
    shapes = list(map(np.shape, chunk_list))
    ndays = shapes[0][0]
    nlons = int(1440 / chunksize)
    nlats = int(720 / chunksize)
    out_array = np.empty((ndays, 720, 1440))
    combs = list(itertools.product(
        list(range(nlats)),
        list(range(nlons))
    ))
    res_list = zip(combs, chunk_list)
    for position, arr in res_list:
        minlon, maxlon = position[0] * chunksize, position[0] * chunksize + chunksize
        minlat, maxlat = position[1] * chunksize, position[1] * chunksize + chunksize
        out_array[:, minlon:maxlon, minlat:maxlat] = arr
    return out_array

In [7]:
# Processing tas yearly
years = list(range(1971, 1972))
vars = ['tasmax', 'tasmin']
for i, year in enumerate(years):
    tasmax_filenames, tasmax_datasets, tasmax_current_year_stack = download_and_stack(year, variable=vars[0])
    tasmin_filenames, tasmin_datasets, tasmin_current_year_stack = download_and_stack(year, variable=vars[1])
    print(tasmax_current_year_stack)
    print(tasmin_current_year_stack)
    
    
    # Finally
    for filename in [*tasmax_filenames, *tasmin_filenames]:
        os.remove(filename)

Starting download pool
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_BNU-ESM_1971.nc


Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/dask/lib/python3.6/multiprocessing/pool.py", li

KeyboardInterrupt: 

In [21]:
year = 1971
year

1971

In [ ]:
tasmax_filenames, tasmax_datasets, tasmax_current_year_stack = download_and_stack(year, variable=vars[0])
tasmin_filenames, tasmin_datasets, tasmin_current_year_stack = download_and_stack(year, variable=vars[1])

Starting download pool
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CCSM4_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CanESM2_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CNRM-CM5_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_GFDL-CM3_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p

In [10]:
tasavg = da.stack((tasmax_current_year_stack, tasmin_current_year_stack), axis = 0)

In [15]:
avg_np = np.mean(tasavg, axis= 0)

In [18]:
avg.shape

(2, 365, 720, 1440)

In [17]:
client.restart()

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787,Cluster Workers: 1 Cores: 4 Memory: 16.81 GB
